Alunos: Giovanna Rodrigues Mendes e Matheus Felipe Alves Durães


RGAs: 2021.1904.032-7 e 2021.1904.008-4

# Descrição:
#### Fazer duas implementações paralelas em CUDA para resolver o problema de calcular as distâncias de todos os caminhos mínimos em um dado grafo de entrada representado por sua matriz de adjacência, como o visto em sala. Você deve implementar duas soluções paralelas distintas para o problema e  analisar cada uma delas em relação a solução sequencial.

#### Você deve comparar o desempenho de as suas soluções com a solução sequencial para grafos com número de vértices entre 512 e 8192  e escrever um relatório com os resultados obtidos. Os códigos fontes juntamente com o relatório em pdf devem ser entregues via AVA.

In [ ]:
#Permite a execução dos códigos abaixo
!pip install git+https://github.com/lesc-ufv/cad4u.git &> /dev/null
!git clone https://github.com/lesc-ufv/cad4u &> /dev/null
%load_ext plugin

In [ ]:
!nvidia-smi

Wed Nov 15 20:11:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Verificação

In [ ]:
%%writefile verify.h

void checkResults(float *cpu, float *gpu, int size) {
  bool pass = true;

  for(int i = 0; i < size; ++i) {
    if(abs(cpu[i] - gpu[i]) > 0.001) {
      printf("FAIL: value of position (%d,%d) on cpu %.2f != %.2f on gpu\n", i/size, i%size, cpu[i], gpu[i]); pass = false; break;
    }
  }

  if (pass) printf("Sucesso! todos os valores foram calculados corretamente!\n");
}

Writing verify.h


## Medição do tempo

In [ ]:
%%writefile time_analisys.h

float elapsed_time;
cudaEvent_t start, stop;                             //# Declara dois eventos

void time_start() {
  cudaEventCreate(&start);                          //# Irá marcar o inicio da execucao
  cudaEventCreate(&stop);                           //# Irá  marcar o final da execucao
  cudaEventRecord(start, 0);                        //# Insere na fila
}

void time_end() {
  cudaEventRecord(stop, 0);                          //# Insere na fila
  cudaEventSynchronize(stop);                        //# Espera terminar
  cudaEventElapsedTime(&elapsed_time, start, stop);  //# Calcula o tempo passado
}

Writing time_analisys.h


## CPU

In [ ]:
%%writefile cpu.h

void cpu_mmatrix(float *hres, float *a, float *b, int n) {
    float temp;
    float auxA;
    float auxB;

    for (int i = 0; i < n; i++) {
        for (int j = 0; j < n; j++) {

            if (i == j) {
                hres[i*n+j] = 0;

            } else if(i > j) {
                hres[i*n+j] = 200;

            } else {
                temp = a[i*n+j];
                for (int k = 0; k < n; k++) {
                    auxA = a[i*n + k];
                    auxB = b[k*n + j];

                    if (temp > (auxA + auxB)) {
                        temp = auxA + auxB;
                    }
                }

                hres[i*n+j] = temp;
            }

        }
    }
}

Writing cpu.h


## GPU: Naive


In [ ]:
%%writefile naive.h

__global__ void matrixMul_naive(float *c, float *a, float *b, int n) {
    // Computa a linha e coluna de cada thread
    int row = blockIdx.x * blockDim.x + threadIdx.x;
    int col = blockIdx.y * blockDim.y + threadIdx.y;

    // Itera sobre a linha e descendo a coluna
    float valorTemp;
    float aux;

    aux = 200;

    if (row == col) {
        c[row * n + col] = 0;
    } else {
        for (int k = 0; k < n; k++) {
          valorTemp = a[row * n + k] + b[k * n + col];
          if(aux > valorTemp) {
              aux = valorTemp;
          }
        }

        c[row * n + col] = aux;
    }

}


Writing naive.h


## GPU: DNS

In [ ]:
%%writefile matriz_DNS.h

__global__ void matrizMul_DNS(float *ans, float *x, float *y, int n) {

    // Calcula o ID unico da thread no bloco
    int t = (blockDim.x * blockDim.y) * threadIdx.z + (threadIdx.y * blockDim.x) + threadIdx.x;

    // Calcula o ID unico do bloco na grid
    int b = (gridDim.x * gridDim.y) * blockIdx.z + (blockIdx.y * gridDim.x) + (blockIdx.x);

    // Tamanho do bloco (Pode ser redundante)
    int tamanhoDoBloco = blockDim.x*blockDim.y*blockDim.z;

    // Tamanho da Grid (Pod ser redundante)
    int tamanhoDaGrid = gridDim.x*gridDim.y*gridDim.z;

    float aux;
    float valorTemp;

    for (int i = b; i < n ; i+= tamanhoDaGrid) {
        for (int j = t; j < n ;j += tamanhoDoBloco) {
            if (i == j) {
                aux = 0;

            } else {
                aux = 200;

                for (int k = 0; k < n ;k++) {
                    valorTemp = x[i * n + k] + y[k * n + j];
                    if (aux > valorTemp) {
                        aux = valorTemp;
                    }
                }
            }
            ans[i * n + j] = aux;
        }
    }
}

Overwriting matriz_mul.h


## Caminho Míınimo: Multiplicação de matrizes

In [ ]:
%%nvprof

#define N 8192  // Matrix NxN

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>
#include "/content/verify.h"
#include "/content/time_analisys.h"
#include "/content/cpu.h"
#include "/content/naive.h"
#include "/content/matriz_DNS.h"

int main(int argc, char* argv[]) {

    int deviceId, numberOfSMs;
    clock_t comeco, fim;

    cudaDeviceProp deviceProp;
    cudaGetDeviceProperties(&deviceProp, 0);
    printf("Dispositivo %d: %s \n", 0, deviceProp.name);
    cudaSetDevice(0);

    size_t bytes = N * N * sizeof(float);

    printf("Ocupação tamanho da Matriz %d\n", bytes);

    float *h_a, *h_cpu, *h_naive, *h_DNS;
    float *d_a, *d_naive, *d_DNS;
    float time_cpu, time_gpu_naive, time_gpu_DNS;

    h_a = (float*) malloc(bytes);
    h_cpu = (float*) malloc(bytes);
    h_naive = (float*) malloc(bytes);
    h_DNS = (float*) malloc(bytes);

    srand(1);
    for (int k = 0; k < N; k++) {
        int aux;
        for (int i = 0; i < N; i++) {
            if (i > k) {
                aux = rand() % 20;

                aux = aux >= 15 ? 200 : aux;

                h_a[i + (k*N)] = aux;

            } else if (i < k) {
                h_a[i + (k*N)] = 200.0;

            } else {
                h_a[i + (k*N)] = 0;
            }

        }
    }

    /* # Print de debug para saber a matriz inicial
    printf("------------------------------------------------\n");

    for (int i = 0; i < (N*N); i++) {
        printf("%3.0f ", h_a[i]);
        if ((i % N) == (N-1)) printf("\n");
    }

    printf("------------------------------------------------\n");
    */

    memset(h_cpu, 0, bytes);
    memset(h_naive, 0, bytes);
    memset(h_DNS, 0, bytes);

    cudaMalloc(&d_a, bytes);
    cudaMalloc(&d_naive, bytes);
    cudaMalloc(&d_DNS, bytes);

    //* Copia os dados para o dispositivo
    if (cudaMemcpy(d_a, h_a, bytes, cudaMemcpyHostToDevice) != 0) {
        printf("Erro ao copiar dados para d_a\n");
    }
    if (cudaMemcpy(d_naive, h_naive, bytes, cudaMemcpyHostToDevice) != 0){
        printf("Erro ao copiar dados para d_naive\n");
    }
    if (cudaMemcpy(d_DNS, h_DNS, bytes, cudaMemcpyHostToDevice) != 0){
        printf("Erro ao copiar dados para d_DNS\n");
    }

    //* Execucao na CPU
    /* # Execução CPU
    comeco = clock();
    cpu_mmatrix(h_cpu, h_a, h_a, N);
    fim = clock() - comeco;
    time_cpu = 1000*((float)fim) / CLOCKS_PER_SEC;
    */

    // Threads por dimensão CTA
    int THREADS = 32; //# Threads aqui!

    // Blocos por dimensão de grid (assume que THREADS vai dividir N igualmente)
    int BLOCKS = N / THREADS;

    // Use dim3 structs for block  and grid dimensions
    dim3 threads(THREADS, THREADS);
    dim3 blocks(BLOCKS, BLOCKS);

    printf("Blocos: %d\nThreads/blocos: %d\nThreads(total): %d\n\n", BLOCKS, THREADS, THREADS*BLOCKS);

    //printf("Tempo de CPU: %15.2lf ms\n", time_cpu);

    //* Chamada do kernel naive
    time_start();
    matrixMul_naive<<<blocks, threads>>>(d_naive, d_a, d_a, N);
    cudaDeviceSynchronize();
    time_end();

    if (cudaMemcpy(h_naive, d_naive, bytes, cudaMemcpyDeviceToHost) != 0) {
        printf("Erro ao copiar resultados de d_naive\n");
    }

    printf("Tempo GPU naive: %12.2lf ms\n", elapsed_time);

    //* Chamada do kernel DNS
    time_start();
    matrizMul_DNS<<<blocks, threads>>>(d_DNS, d_a, d_a, N);
    cudaDeviceSynchronize();
    time_end();

    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
        printf("CUDA error: %s\n" ,cudaGetErrorString(err));
        exit(1);
    }

    if (cudaMemcpy(h_DNS, d_DNS, bytes, cudaMemcpyDeviceToHost) != 0) {
        printf("Erro ao copiar resultados de d_DNS\n");
        exit(1);
    }

    printf("Tempo GPU DNS: %7.2lf ms\n", elapsed_time);

    /*
    printf("------------------------------------------\nCPU: \n");

    for (int i = 0; i < (N*N); i++) {
        printf("%3.0f ", h_cpu[i]);
        if ((i % N) == (N-1)) printf("\n");
    }

    printf("------------------------------------------\nGPU Naive: \n");

    for (int i = 0; i < (N*N); i++) {
        printf("%3.0f ", h_naive[i]);
        if ((i % N) == (N-1)) printf("\n");
    }

    printf("------------------------------------------\nGPU DNS: \n");

    for (int i = 0; i < (N*N); i++) {
        printf("%3.0f ", h_DNS[i]);
        if ((i % N) == (N-1)) printf("\n");
    }
    */

    //# Checkresults desligados para testar maiores valores de N sem a CPU
    //checkResults(h_cpu, h_naive, N*N);
    //checkResults(h_cpu, h_DNS, N*N);

    free(h_cpu); free(h_naive); free(h_a); free(h_DNS);
    cudaFree(d_naive); cudaFree(d_a); cudaFree(d_DNS);
    return 0;
}

==26125== NVPROF is profiling process 26125, command: /content/code.out
Dispositivo 0: Tesla T4 
Ocupação tamanho da Matriz 268435456
Blocos: 256
Threads/blocos: 32
Threads(total): 8192

Tempo GPU naive:     18304.20 ms
Tempo GPU caminhoMin: 9518.98 ms
==26125== Profiling application: /content/code.out
==26125== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   65.13%  18.3037s         1  18.3037s  18.3037s  18.3037s  matrixMul_naive(float*, float*, float*, int)
                   33.87%  9.51868s         1  9.51868s  9.51868s  9.51868s  matrizMul_caminhoMin(float*, float*, float*, int)
                    0.61%  170.24ms         3  56.746ms  55.621ms  57.722ms  [CUDA memcpy HtoD]
                    0.40%  112.19ms         2  56.095ms  55.826ms  56.364ms  [CUDA memcpy DtoH]
      API calls:   98.50%  27.8224s         2  13.9112s  9.51870s  18.3037s  cudaDeviceSynchronize
                    1.01%  283.89ms         5 